In [26]:
import QuantLib as ql

# Dagens dato og settings
today = ql.Date(25, 5, 2025)
ql.Settings.instance().evaluationDate = today

# Swaprenter (til 15 år)
rates = [
    (1, 0.0205), (2, 0.020879), (3, 0.022006), (4, 0.023196), (5, 0.024289),
    (6, 0.025137), (7, 0.025853), (8, 0.026505), (9, 0.027091),
    (10, 0.027606), (15, 0.0295)
]
curve_dates = [today + ql.Period(y, ql.Years) for (y, _) in rates]
curve_rates = [r for (_, r) in rates]

# Opret yield-kurve
zero_curve = ql.ZeroCurve(curve_dates, curve_rates, ql.Actual365Fixed())
yield_curve_handle = ql.YieldTermStructureHandle(zero_curve)

# Hull-White procesparametre
a = 0.03
sigma = 0.01
process = ql.HullWhiteProcess(yield_curve_handle, a, sigma)

# Simulering 10 år frem med daglige trin
time_horizon = 10  # år
steps = 10       # ca. daglige trin

# Random generator
rng = ql.GaussianRandomSequenceGenerator(
    ql.UniformRandomSequenceGenerator(steps, ql.UniformRandomGenerator())
)
seq = ql.GaussianPathGenerator(process, time_horizon, steps, rng, False)

# Simuler én sti
path = seq.next().value()
time_grid = seq.timeGrid()
rates = [path[i] for i in range(len(time_grid))]

# Udskriv: tid i år og rente
for t, r in zip(time_grid, rates):
    print(f"Tid: {t:.2f} år – rente: {r:.4%}")


Tid: 0.00 år – rente: 2.0500%
Tid: 1.00 år – rente: 3.7429%
Tid: 2.00 år – rente: 4.1104%
Tid: 3.00 år – rente: 3.0417%
Tid: 4.00 år – rente: 1.8339%
Tid: 5.00 år – rente: 1.3709%
Tid: 6.00 år – rente: 2.4573%
Tid: 7.00 år – rente: 2.2782%
Tid: 8.00 år – rente: 2.7961%
Tid: 9.00 år – rente: 3.0189%
Tid: 10.00 år – rente: 3.0916%


In [132]:
import QuantLib as ql
import numpy as np

# 1. Parametre og dato
today = ql.Date(25, 5, 2025)
ql.Settings.instance().evaluationDate = today
day_count = ql.Actual365Fixed()
calendar = ql.TARGET()

# 2. Yield curve (15 års zero curve) og tillad extrapolation
rates = [
    (1, 0.0205), (2, 0.020879), (3, 0.022006), (4, 0.023196), (5, 0.024289),
    (6, 0.025137), (7, 0.025853), (8, 0.026505), (9, 0.027091),
    (10, 0.027606), (15, 0.0295), (30, 0.0282)
]
dates = [today + ql.Period(y, ql.Years) for y, _ in rates]
zero_curve = ql.ZeroCurve(dates, [r for _, r in rates], day_count)
zero_curve.enableExtrapolation()                  # <<— tillad extrapolation
yc_handle = ql.YieldTermStructureHandle(zero_curve)

# 3. Hull-White-processen
a, sigma = 0.009, 0.022
hw_process = ql.HullWhiteProcess(yc_handle, a, sigma)

# 4. Simpel prepayment-model (CPR afhænger af r - kupon)))
def cpr_model(rate, balance, remaining_months, current_payment,
              base_cpr=0.02, alpha=0.5, max_cpr=0.15):
    # ny månedlig rente
    r_m = rate / 12.0
    # beregn ny refinancing-ydelse
    if remaining_months > 0 and r_m > 0:
        new_payment = (r_m * balance) / (1 - (1 + r_m) ** (-remaining_months))
    else:
        new_payment = current_payment
    # procentvis besparelse
    delta = max(0.0, (current_payment - new_payment) / current_payment)
    # CPR afhænger af base + alpha * besparelse
    cpr = base_cpr + alpha * delta
    # hold CPR inden for [base, max_cpr]
    return min(max_cpr, max(base_cpr, cpr))

# 5. MBS-parametre
face_value   = 1_000_000    # Hovedstol
coupon_rate  = 0.035       # 3% årligt
term_months  = 10*12          # 30 år
monthly_rate = coupon_rate / 12

# Manuel beregning af månedlig ydelse
monthly_payment = (monthly_rate * face_value) / (1 - (1 + monthly_rate) ** (-term_months))

# 6. Monte Carlo-simulering
num_paths      = 1000
steps_per_year = 12
time_horizon   = term_months / 12       # 30 år
steps          = int(time_horizon * steps_per_year)
dt             = 1.0 / steps_per_year

rng = ql.GaussianRandomSequenceGenerator(
    ql.UniformRandomSequenceGenerator(steps, ql.UniformRandomGenerator()))
seq = ql.GaussianPathGenerator(hw_process, time_horizon, steps, rng, False)

discount_curve = yc_handle.currentLink()
npv_paths = []

for _ in range(num_paths):
    path = seq.next().value()
    rates_along_path = [path[i] for i in range(len(path))]

    balance = face_value
    npv = 0.0
    for m in range(term_months):
        if balance <= 0:
            break

         # antal måneder tilbage
        remaining = term_months - m
    
        # tid i år og rente
        t    = (m + 1) * dt
        idx  = min(int(t * steps_per_year) - 1, len(rates_along_path) - 1)
        rate = rates_along_path[idx]
    
        # NYE CPR baseret på refinance-besparelse
        cpr = cpr_model(
            rate=rate,
            balance=balance,
            remaining_months=remaining,
            current_payment=monthly_payment,
            base_cpr=0.02,
            alpha=0.5,
            max_cpr=0.30
        )
        smm = 1 - (1 - cpr) ** (1/12)
    
        # cashflows som før...
        interest   = balance * monthly_rate
        principal  = monthly_payment - interest
        prepayment = smm * (balance - principal)
        cf         = interest + principal + prepayment
        balance   -= (principal + prepayment)
    
        # diskonter
        df   = np.exp(-dt * sum(rates_along_path[:idx+1]))
        npv += cf * df

    npv_paths.append(npv)

# 7. Estimeret OAS-justeret MBS-værdi
avg_npv = np.mean(npv_paths)
print(f"Estimeret MBS-værdi (OAS-justeret): {avg_npv:,.2f} DKK")


Estimeret MBS-værdi (OAS-justeret): 1,044,199.53 DKK


In [133]:
def prepaymentrate(r, )

AttributeError: module 'numpy' has no attribute 'pmt'